In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
import gensim

In [ ]:
df =  pd.read_csv(r'C:\Users\PRATIK KOTKAR\Desktop\Sentimental_Churn\sentiment.tsv',delimiter='\t',names= ['class','tweet'])

In [ ]:
df.head()

In [ ]:
df['class'].replace('pos',0, inplace=True)
df['class'].replace('neg',1, inplace=True)
df.head()

In [ ]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])","",tweet.lower()).split())

In [ ]:
df['clean_tweet']=df['tweet'].apply(process_tweet)

In [ ]:
df.head()

In [ ]:
del df['tweet']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df,df["class"], test_size = 0.3, random_state = 0)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
## for transforming the 80% of the train data ##
X_train_counts = count_vect.fit_transform(X_train)
X_train_tfidf = transformer.fit_transform(X_train_counts)
## for transforming the 20% of the train data which is being used for testing ##
x_test_counts = count_vect.transform(X_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train_tfidf,y_train)

In [ ]:
predictions = model.predict(x_test_tfidf)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=200)
model.fit(X_train_counts,y_train)
predictions = model.predict(x_test_counts)

In [ ]:
roc_auc_score(y_test, predictions)

In [ ]:
def get_scores(model,x_train, x_test, y_train, y_test):
    model.fit(x_train,y_train)
    return model.score(x_test,y_test)

In [ ]:
get_scores(RandomForestClassifier,(X_train_tfidf,x_test_tfidf,y_train,y_test))

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10)

In [ ]:
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()
lreg.fit(X_train_tfidf,y_train) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier